# 7. 다층 퍼셉트론(MultiLayer Perceptron, MLP)으로 텍스트 분류하기

# 1) 다층 퍼셉트론(MultiLayer Perceptron, MLP)
> 은닉층이 1개 이상 추가된 신경망

## 2) 케라스의 texts_to_matrix() 이해하기
- **texts_to_matrix()** : 입력된 텍스트 데이터로부터 행렬(matrix)를 만드는 도구
    - 'binary', 'count', 'freq', 'tfidf'로 총 4개의 모드를 지원
    - **count** : DTM 생성. 인덱스 0은 할당이 안되서 모든 경우에 다 0으로 처리
    - **** : 

In [1]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer

texts = ['먹고 싶은 사과', '먹고 싶은 바나나', 
         '길고 노란 바나나 바나나', '저는 과일이 좋아요']

t = Tokenizer()
t.fit_on_texts(texts)
print(t.word_index)

{'바나나': 1, '먹고': 2, '싶은': 3, '사과': 4, '길고': 5, '노란': 6, '저는': 7, '과일이': 8, '좋아요': 9}


In [2]:
# count 모드는 DTM 생성 --> 단어 순서 정보는 보존X
print(t.texts_to_matrix(texts, mode = 'count'))

[[0. 0. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 2. 0. 0. 0. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 1.]]


In [3]:
# binary 모드 --> count 모드와 다른 점은 해당 단어가 있는지 없는지임
print(t.texts_to_matrix(texts, mode = 'binary'))

[[0. 0. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 1.]]


In [4]:
# 앞에서의 TF-IDF 와는 다른 식을 사용함
print(t.texts_to_matrix(texts, mode = 'tfidf').round(2))

[[0.   0.   0.85 0.85 1.1  0.   0.   0.   0.   0.  ]
 [0.   0.85 0.85 0.85 0.   0.   0.   0.   0.   0.  ]
 [0.   1.43 0.   0.   0.   1.1  1.1  0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   1.1  1.1  1.1 ]]


In [5]:
# (각 문서에서의 각 단어의 등장 횟수)/(각 문서마다 등장한 모든 단어 개수의 총 합)
print(t.texts_to_matrix(texts, mode = 'freq').round(2))

[[0.   0.   0.33 0.33 0.33 0.   0.   0.   0.   0.  ]
 [0.   0.33 0.33 0.33 0.   0.   0.   0.   0.   0.  ]
 [0.   0.5  0.   0.   0.   0.25 0.25 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.33 0.33 0.33]]


## 3) 20개 뉴스 그룹(Twenty Newsgroups) 데이터에 대한 이해

In [6]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

# 'train'을 기재하면 훈련 데이터만 리턴
# 'all'을 기재하면 전체 18846개 샘플 리턴
# 'test'를 넣으면 테스트 데이터 리턴
newsdata = fetch_20newsgroups(subset = 'train')

print(newsdata.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [21]:
print('훈련용 샘플의 개수 : {}'.format(len(newsdata.data)))

print('총 주제의 개수 : {}'.format(len(newsdata.target_names)))
print(newsdata.target_names)

훈련용 샘플의 개수 : 11314
총 주제의 개수 : 20
['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [22]:
print('첫번째 샘플의 레이블 : {}'.format(newsdata.target[0]))

첫번째 샘플의 레이블 : 7


In [1]:
print('7번 레이블이 의미하는 주제 : {}'.format(newsdata.target_names[7]))

NameError: name 'newsdata' is not defined